# Проект для «Викишоп» с BERT

Интернет-магазин «Викишоп» запускает новый сервис. Теперь пользователи могут редактировать и дополнять описания товаров, как в вики-сообществах. То есть клиенты предлагают свои правки и комментируют изменения других. Магазину нужен инструмент, который будет искать токсичные комментарии и отправлять их на модерацию. 

Обучите модель классифицировать комментарии на позитивные и негативные. В вашем распоряжении набор данных с разметкой о токсичности правок.

Постройте модель со значением метрики качества *F1* не меньше 0.75. 

**Инструкция по выполнению проекта**

1. Загрузите и подготовьте данные.
2. Обучите разные модели. 
3. Сделайте выводы.

Для выполнения проекта применять *BERT* необязательно, но вы можете попробовать.

**Описание данных**

Данные находятся в файле `toxic_comments.csv`. Столбец *text* в нём содержит текст комментария, а *toxic* — целевой признак.

## Подготовка

In [2]:
# отключим предупреждающие уведомления
import warnings
warnings.filterwarnings('ignore')

# Добавим основные библиотеки
import pandas as pd
import numpy as np
import os.path
import re

# добавим библиотеки для работы с текстами
import nltk
nltk.download('punkt')
nltk.download('wordnet')
nltk.download('stopwords')
from sklearn.feature_extraction.text import TfidfVectorizer

# библиотеки для работы с BERT 
import torch
import transformers as ppb

# Добавим библиотеки для отрисовки графиков
import matplotlib.pyplot as plt
plt.style.use('dark_background')

# добавим необходимые библиотеки для построения моделей
from imblearn.pipeline import Pipeline, make_pipeline
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.linear_model import LogisticRegression
from sklearn.svm import LinearSVC
from sklearn.metrics import f1_score

# Сделаем небольшие настройки pandas для комфортного отображения проекта
pd.set_option('display.max_columns', None)
pd.set_option('display.float_format', lambda x: '%.4f' % x)
pd.options.mode.chained_assignment = None

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\PSImera\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\PSImera\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\PSImera\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [3]:
# Импортируем датасеты методом try/except и установим индексом столбец date
try:
    path_df = os.path.join('datasets/toxic_comments.csv') 
    df = pd.read_csv(path_df)
except:
    df = pd.read_csv('https://code.s3.yandex.net/datasets/toxic_comments.csv')

In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 159292 entries, 0 to 159291
Data columns (total 3 columns):
 #   Column      Non-Null Count   Dtype 
---  ------      --------------   ----- 
 0   Unnamed: 0  159292 non-null  int64 
 1   text        159292 non-null  object
 2   toxic       159292 non-null  int64 
dtypes: int64(2), object(1)
memory usage: 3.6+ MB


In [5]:
display(df.head(5))
display(df.tail(5))

,Unnamed: 0,text,toxic
0,0,Explanation\r\nWhy the edits made under my use...,0
1,1,D'aww! He matches this background colour I'm s...,0
2,2,"Hey man, I'm really not trying to edit war. It...",0
3,3,"""\r\nMore\r\nI can't make any real suggestions...",0
4,4,"You, sir, are my hero. Any chance you remember...",0


,Unnamed: 0,text,toxic
159287,159446,""":::::And for the second time of asking, when ...",0
159288,159447,You should be ashamed of yourself \r\n\r\nThat...,0
159289,159448,"Spitzer \r\n\r\nUmm, theres no actual article ...",0
159290,159449,And it looks like it was actually you who put ...,0
159291,159450,"""\r\nAnd ... I really don't think you understa...",0


Столбец `Unnamed: 0` похож на столбец индексов, но последние его значения не совпадают с количеством значений датасета, а следовательно там содержатся пропуски. Эти старые индексы нам не понадобятся, удалим этот столбец

In [6]:
df.drop(['Unnamed: 0'], axis=1, inplace=True)

In [7]:
toxic_percent = round(df['toxic'].mean()*100, 2) 
print(f'В датасете {toxic_percent}% токсичный комментариев')

В датасете 10.16% токсичный комментариев


Дисбаланс классов весьма значительный. Разобивать датасет на выборки стоит с учётом этого дисбаланса

In [9]:
train, test = train_test_split(df, test_size=0.2, random_state=1337, stratify=df['toxic'])

# проверяем разбивку с учётом дисбаланса
train_toxic_percent = round(train['toxic'].mean()*100, 2)
test_toxic_percent = round(test['toxic'].mean()*100, 2) 
print(f'В тренировочной выборке {train_toxic_percent}% токсичный комментариев')
print(f'В тестовой выборке {test_toxic_percent}% токсичный комментариев')

В тренировочной выборке 10.16% токсичный комментариев
В тестовой выборке 10.16% токсичный комментариев


Выделим целевой признак

In [17]:
def X_y_split(df, y):
    X = df.drop([y], axis=1)
    y = df[y]
    return X, y

X_train, y_train, = X_y_split(train, 'toxic')
X_test, y_test = X_y_split(test, 'toxic')

### Лемматизация

In [18]:
lemmatizer = nltk.stem.WordNetLemmatizer()
def lemmatization(text):
    tokenized = nltk.tokenize.word_tokenize(text)
    lemmatized = ' '.join([lemmatizer.lemmatize(x) for x in tokenized])
    result = ' '.join(re.sub(r'[^a-z]', ' ',lemmatized.lower()).split())
    return result

X_train['lemmatized'] = X_train['text'].apply(lemmatization)
X_test['lemmatized'] = X_test['text'].apply(lemmatization)

### Векторизация

In [19]:
# stop_words = set(nltk.corpus.stopwords.words('english'))
stop_words = nltk.corpus.stopwords.words('english')

tfidf_vectorizer = TfidfVectorizer(stop_words=stop_words, ngram_range=(1,2))

X_vec_train = tfidf_vectorizer.fit_transform(X_train['lemmatized'])
X_vec_test = tfidf_vectorizer.transform(X_test['lemmatized'])

### Подготовим данные для BERT модели"

In [20]:
# загрузка предобученной модели и токенизатора
tokenizer = ppb.AutoTokenizer.from_pretrained('unitary/toxic-bert')
bert_model = ppb.AutoModelForSequenceClassification.from_pretrained('unitary/toxic-bert')

# токенезируем тексты
tokenized = df['text'].apply(lambda x: tokenizer.encode(x, add_special_tokens=True, padding=True, truncation=True, max_length=512))

# определим максимальную длинну вектора после токенизации
max_len = max([len(v) for v in tokenized.values])

# увеличим вектора до длинны вмаксимального
padded = np.array([i + [0]*(max_len - len(i)) for i in tokenized.values])

# создадим маску важных токенов
attention_mask = np.where(padded != 0, 1, 0)

tokenizer_config.json:   0%|          | 0.00/174 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/811 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/438M [00:00<?, ?B/s]

In [21]:
# создадим девайс для расчетов с помощью CUDA если такая возможность есть
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
bert_model.to(device)

# в качестве размера батча возьмём максимальный кратный делитель датасета, но не более заданного параметра
max_batch_size = 100
batch_size = max([r for r in [x for x in range(1, df.shape[0]+1) if df.shape[0] % x == 0] if r <= max_batch_size])

embeddings = []
for i in range(padded.shape[0] // batch_size):
        batch = torch.tensor(padded[batch_size*i:batch_size*(i+1)]).to(device) 
        mask = torch.tensor(attention_mask[batch_size*i:batch_size*(i+1)]).to(device) 
        
        with torch.no_grad():
            bert_model.eval()
            batch_embeddings = bert_model(batch, attention_mask=mask)
        
        embeddings.append(batch_embeddings[0].detach().cpu().numpy())

X_bert = np.concatenate(embeddings)

Создадим выборки для обучения моделей после эмбеддинга BERT

In [22]:
X_bert_train, X_bert_test = train_test_split(X_bert, test_size=0.2, random_state=1337, stratify=df['toxic'])

## Обучение

In [23]:
results = pd.DataFrame(columns=['Модель', 'F1 Метрика'])

### LogisticRegression

Используем кроссвалидацию для поиска параметров с помощью `GridSearchCV`

In [24]:
lr_parameters = {'clf__C': [0.1, 0.2, 0.4, 0.6, 0.8, 1.0, 2.0, 4.0, 9.0]}

lr_pipe = Pipeline([('vect', TfidfVectorizer(stop_words=stop_words, ngram_range=(1,2))),
                    ('clf', LogisticRegression(random_state=1337, class_weight='balanced', max_iter=1000))])

lr_grsch = GridSearchCV(lr_pipe,
                        lr_parameters,
                        cv=5,
                        scoring='f1',
                        verbose=10
                        )

lr_grsch.fit(X_train['lemmatized'], y_train)

results.loc[0] = ['LogisticRegression CV', lr_grsch.best_score_]
print('LogisticRegression')
print(f'Наилучший показатель f1 на кросс-валидации : {round(lr_grsch.best_score_, 4)}')
print(f'при параметрах: {lr_grsch.best_params_}')

Fitting 5 folds for each of 9 candidates, totalling 45 fits
[CV 1/5; 1/9] START clf__C=0.1..................................................
[CV 1/5; 1/9] END ...................clf__C=0.1;, score=0.686 total time=  24.0s
[CV 2/5; 1/9] START clf__C=0.1..................................................
[CV 2/5; 1/9] END ...................clf__C=0.1;, score=0.689 total time=  22.4s
[CV 3/5; 1/9] START clf__C=0.1..................................................
[CV 3/5; 1/9] END ...................clf__C=0.1;, score=0.678 total time=  21.8s
[CV 4/5; 1/9] START clf__C=0.1..................................................
[CV 4/5; 1/9] END ...................clf__C=0.1;, score=0.687 total time=  21.4s
[CV 5/5; 1/9] START clf__C=0.1..................................................
[CV 5/5; 1/9] END ...................clf__C=0.1;, score=0.682 total time=  21.4s
[CV 1/5; 2/9] START clf__C=0.2..................................................
[CV 1/5; 2/9] END ...................clf__C=0.2;,

In [25]:
lr_bert_parameters = {'C': [0.1, 0.2, 0.4, 0.6, 0.8, 1.0, 2.0, 4.0, 9.0]}

lr_bert_model = LogisticRegression(max_iter=1000, random_state=1337, class_weight='balanced')
lr_bert_grsch = GridSearchCV(lr_bert_model,
                             lr_bert_parameters,
                             cv=5,
                             scoring='f1',
                             n_jobs=-1,
                             verbose=10
                             )

lr_bert_grsch.fit(X_bert_train, y_train)

results.loc[1] = ['LogisticRegression BERT CV', lr_bert_grsch.best_score_]
print('LogisticRegression BERT')
print(f'Наилучший показатель f1 на кросс-валидации : {round(lr_bert_grsch.best_score_, 4)}')
print(f'при параметрах: {lr_bert_grsch.best_params_}')

Fitting 5 folds for each of 9 candidates, totalling 45 fits
LogisticRegression BERT
Наилучший показатель f1 на кросс-валидации : 0.908
при параметрах: {'C': 9.0}


### LinearSVC

In [26]:
svc_parameters = {'clf__C': np.linspace(0.1, 1, num=10, endpoint=True)}

svc_pipe = Pipeline([('vect', TfidfVectorizer(stop_words=stop_words, ngram_range=(1,2))),
                    ('clf', LinearSVC(random_state=1337, class_weight='balanced'))])

svc_model = LinearSVC(random_state=1337, class_weight='balanced')
svc_grsch = GridSearchCV(svc_pipe,
                         svc_parameters,
                         cv=5,
                         scoring='f1',
                         verbose=10
                         )

svc_grsch.fit(X_train['lemmatized'], y_train)

results.loc[2] = ['LinearSVC CV', svc_grsch.best_score_]
print('LinearSVC')
print(f'Наилучший показатель f1 на кросс-валидации : {round(svc_grsch.best_score_, 4)}')
print(f'при параметрах: {svc_grsch.best_params_}')

Fitting 5 folds for each of 10 candidates, totalling 50 fits
[CV 1/5; 1/10] START clf__C=0.1.................................................
[CV 1/5; 1/10] END ..................clf__C=0.1;, score=0.762 total time=  20.2s
[CV 2/5; 1/10] START clf__C=0.1.................................................
[CV 2/5; 1/10] END ..................clf__C=0.1;, score=0.751 total time=  19.8s
[CV 3/5; 1/10] START clf__C=0.1.................................................
[CV 3/5; 1/10] END ..................clf__C=0.1;, score=0.740 total time=  19.8s
[CV 4/5; 1/10] START clf__C=0.1.................................................
[CV 4/5; 1/10] END ..................clf__C=0.1;, score=0.744 total time=  19.8s
[CV 5/5; 1/10] START clf__C=0.1.................................................
[CV 5/5; 1/10] END ..................clf__C=0.1;, score=0.746 total time=  21.1s
[CV 1/5; 2/10] START clf__C=0.2.................................................
[CV 1/5; 2/10] END ..................clf__C=0.2;

In [27]:
svc_bert_parameters = {'C': np.linspace(0.1, 1, num=10, endpoint=True)}

svc_bert_model = LinearSVC(random_state=1337, class_weight='balanced')
svc_bert_grsch = GridSearchCV(svc_bert_model,
                              svc_bert_parameters,
                              cv=5,
                              scoring='f1',
                              verbose=3
                              )

svc_bert_grsch.fit(X_bert_train, y_train)

results.loc[3] = ['LinearSVC BERT CV', svc_bert_grsch.best_score_]
print('LinearSVC BERT')
print(f'Наилучший показатель f1 на кросс-валидации : {round(svc_bert_grsch.best_score_, 4)}')
print(f'при параметрах: {svc_bert_grsch.best_params_}')

Fitting 5 folds for each of 10 candidates, totalling 50 fits
[CV 1/5] END .............................C=0.1;, score=0.910 total time=   1.8s
[CV 2/5] END .............................C=0.1;, score=0.902 total time=   1.9s
[CV 3/5] END .............................C=0.1;, score=0.904 total time=   1.9s
[CV 4/5] END .............................C=0.1;, score=0.907 total time=   1.9s
[CV 5/5] END .............................C=0.1;, score=0.908 total time=   1.9s
[CV 1/5] END .............................C=0.2;, score=0.911 total time=   2.0s
[CV 2/5] END .............................C=0.2;, score=0.904 total time=   2.1s
[CV 3/5] END .............................C=0.2;, score=0.905 total time=   2.2s
[CV 4/5] END .............................C=0.2;, score=0.908 total time=   2.2s
[CV 5/5] END .............................C=0.2;, score=0.908 total time=   2.3s
[CV 1/5] END .............C=0.30000000000000004;, score=0.916 total time=   2.8s
[CV 2/5] END .............C=0.30000000000000004;

In [28]:
results

,Модель,F1 Метрика
0,LogisticRegression CV,0.7747
1,LogisticRegression BERT CV,0.9080
2,LinearSVC CV,0.7874
3,LinearSVC BERT CV,0.9270


## Проверим работу модели с найденными параметрами на тестовой выборке

In [29]:
svc_bert_predict = svc_bert_grsch.best_estimator_.predict(X_bert_test)

svc_bert_f1_score = f1_score(y_test, svc_bert_predict)

results.loc[7] = ['LinearSVC BERT', svc_bert_f1_score]
print('LinearSVC BERT', round(svc_bert_f1_score, 4))

LinearSVC BERT 0.9331


## Выводы

Использовано два метода предподготовки данных векторизация `tf-idf` и `toxic-bert`. Так же использовано две модели `LogisticRegression` и `LinearSVC`. Кроссвалидацией при помощи `GridSearchCV` найдены параметры для разных комбинаций векторизации и модели, лучший результат на кроссвалидации показала `LinearSVC` модель после BERT векторизации с показателем 0.9247. На тестовой выборке модель показала результат F1 = 0.9331